In [1]:
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional,TimeDistributed,Input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

Using TensorFlow backend.


In [0]:
import re

In [3]:
from google.colab import files
upload=files.upload()


Saving jhon_maxwell_trad.csv to jhon_maxwell_trad.csv


In [0]:
data=pd.read_csv('jhon_maxwell_trad.csv')

In [0]:
data=data[['español','ingles']]

In [7]:
data.head()

,español,ingles
0,b'AUTOR DE MAYOR VENTAS DEL NEW YORK LIMES JO...,B NEW WINTER SALES ENVELOPE LIMAS JOHN C.MAXWE...
1,E S I ) E I. L l D E R A Z G O iVIVE TU SUEÑ...,E S I) E I. L l D E R A Z G O LIVE YOUR DREAM!...
2,le he preguntado a cientos,I've asked hundreds
3,por no decir miles,not to say thousands
4,de personas en cuanto a sus sueños. Algunos l...,of people regarding their dreams. Some describ...


In [0]:
data['español']=data['español'].apply(lambda x: re.sub('[^a-zA-z0-9Ñ-ñ\s]','',x))

In [0]:
data['ingles']=data['ingles'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

In [0]:
data['ingles']=data['ingles'].apply(lambda x: x.lower())
data['español']=data['español'].apply(lambda x: x.lower())

In [11]:
data.head(10)

,español,ingles
0,bautor de mayor ventas del new york limes joh...,b new winter sales envelope limas john cmaxwel...
1,e s i e i l l d e r a z g o ivive tu sueño ...,e s i e i l l d e r a z g o live your dream 10...
2,le he preguntado a cientos,ive asked hundreds
3,por no decir miles,not to say thousands
4,de personas en cuanto a sus sueños algunos lo...,of people regarding their dreams some describe...
5,o alguien en quien desearian transformarse si...,or someone in whom they would like to transfor...
6,no saben como llegar de aqui a alla si eso te...,they do not know how to get from here to there...
7,vii cual es tu sueño entonces te alegrara...,vii what is your dream then youll be glad to k...
8,y creo que este libro te ayudara conoce las ...,and i think this book will help you know the a...
9,recuerdas que alguna vez el maestro hiciera ...,do you remember that once the teacher did a re...


In [0]:
x=data['español']
y=data['ingles']

In [13]:
caracteres_x=376143
palabras_x=72091
palabras_y=66298
caracteres_y=349659
ejemplos_x_e_y=2961
#y = trad en ing x = orig en esp
x.shape,y.shape


((2961,), (2961,))

In [0]:
pp=set()
for i in y:
    for k in i.split(" "):
        pp.add(k)

In [15]:
len(pp)

6000

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
tokenx=Tokenizer(9095)
tokeny=Tokenizer(5961)

In [0]:
tokenx.fit_on_texts(x)
tokeny.fit_on_texts(y)

In [0]:
x=np.asarray(tokenx.texts_to_sequences(x.values))
y=np.asarray(tokeny.texts_to_sequences(y.values))

In [0]:
padx=pad_sequences(x)
pady=pad_sequences(y)

In [0]:
todo_y=[]
todo_y.extend(tokeny.index_word.keys())

In [0]:
todo_y=np.asarray(todo_y)

In [0]:
todo_y=np.asarray(todo_y).reshape(todo_y.shape[0],-1)

In [24]:
padx.shape,pady.shape,todo_y.shape

((2961, 299), (2961, 301), (5951, 1))

In [0]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [26]:
#input encoder
entrada=Input(shape=(None,))
#embedding
embedding=Embedding(9095,320,input_shape=(301,))(entrada)
#regularizacion
dropout=TimeDistributed(Dropout(0.33))(embedding)
#lstm
bidi_lstm=LSTM(150,return_sequences=True,return_state=True,recurrent_dropout=0.25)(dropout)
#output encoder
output_layer,state_h,state_c=bidi_lstm
estates=[state_h,state_c]

#input decoder
entrada2=Input(shape=(None,))
#embedding2
embedding2=Embedding(6001,320,input_shape=(301,))(entrada2)
#reguralizacion 2
dropout2=TimeDistributed(Dropout(0.35))(embedding2)
#lstm2
lstm1=LSTM(150,return_sequences=False)
lstm2=lstm1(dropout2,initial_state=estates)

#salida decoder
output=Dense(5951,activation='softmax')
salida=output(lstm2)

model=Model([entrada,entrada2],salida)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])#, options = run_opts)

W0719 13:32:59.819703 140613770319744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0719 13:32:59.864937 140613770319744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0719 13:32:59.873753 140613770319744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0719 13:32:59.899891 140613770319744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0719 13:32:59.911551 

In [27]:

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 320)    2910400     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 320)    1920320     input_2[0][0]                    
__________________________________________________________________________________________________
time_distr

In [0]:
def onehot(matrix,dim=2961):
    n_matrix=np.zeros((dim,len(matrix[0])+1))
    for i,matrix in enumerate(matrix):
        n_matrix[i,matrix]=1
    return n_matrix

In [0]:
p=set()
for i in y:
#   print(i)
  for k in i:
    p.add(k)

In [0]:
pl=[]
for i in p:
  pl.append(i)

In [0]:
pl=np.asarray(pl)

In [0]:
pltc=np.zeros([2961,5951])

In [0]:
pl[0]
for i in range(pltc.shape[0]):
  pltc[i]=pl

In [0]:
maxx=0
maxy=0
maxc=0

In [0]:
pltc=pltc/5951.0

In [53]:
print(maxx,maxy,maxc)

9027 5951 5951.0


In [79]:
for i in pltc[0]:
  print(i)

0.00016803898504453034
0.0003360779700890607
0.000504116955133591
0.0006721559401781214
0.0008401949252226517
0.001008233910267182
0.0011762728953117123
0.0013443118803562427
0.001512350865400773
0.0016803898504453034
0.0018484288354898336
0.002016467820534364
0.0021845068055788945
0.0023525457906234245
0.002520584775667955
0.0026886237607124854
0.0028566627457570154
0.003024701730801546
0.0031927407158460763
0.0033607797008906068
0.003528818685935137
0.0036968576709796672
0.0038648966560241977
0.004032935641068728
0.004200974626113258
0.004369013611157789
0.004537052596202319
0.004705091581246849
0.00487313056629138
0.00504116955133591
0.00520920853638044
0.005377247521424971
0.005545286506469501
0.005713325491514031
0.005881364476558562
0.006049403461603092
0.006217442446647623
0.006385481431692153
0.006553520416736683
0.0067215594017812135
0.0068895983868257436
0.007057637371870274
0.0072256763569148045
0.0073937153419593345
0.0075617543270038645
0.007729793312048395
0.0078978322970

In [0]:
historial=model.fit([padx,pady],pltc,epochs=7,batch_size=105)

Epoch 1/7
2961/2961 [==============================] - 127s 43ms/step - loss: 25290.9202 - acc: 0.2340
Epoch 2/7
2961/2961 [==============================] - 127s 43ms/step - loss: 25299.7532 - acc: 0.2259
Epoch 3/7
2961/2961 [==============================] - 127s 43ms/step - loss: 25298.3005 - acc: 0.0963
Epoch 4/7
2961/2961 [==============================] - 127s 43ms/step - loss: 25298.3158 - acc: 0.0932
Epoch 5/7
 630/2961 [=====>........................] - ETA: 1:39 - loss: 25298.1768 - acc: 0.3333